<a href="https://colab.research.google.com/github/andohyung/MS_Project/blob/main/%EB%85%BC%EB%AC%B8%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%B6%94%EC%B6%9C_TFIDF%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

한국어 키워드

In [ ]:
pip install numpy scikit-learn konlpy PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.1 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import PyPDF2

# KONLPY 설치 코드 (주석 처리하여 참고용으로 제공)
# !pip install konlpy

# PDF 파일에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# PDF 파일 경로 설정
pdf_path = '/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/일일 착용 콘택트렌즈의 연속 착용에 따른 세균 오염.pdf'

# PDF 파일에서 텍스트 추출
korean_text = extract_text_from_pdf(pdf_path)

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    return text

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    okt = Okt()
    nouns = okt.nouns(text)
    return [' '.join(nouns)]

# 텍스트 전처리
preprocessed_text = preprocess_text(korean_text)

# 명사만 추출
nouns_text = extract_nouns(preprocessed_text)

# TF-IDF 벡터라이저 초기화 및 키워드 추출 (ngram_range=(1, 2)로 설정하여 unigram과 bigram을 모두 고려)
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(nouns_text)

# TF-IDF 결과를 가져와서 높은 순으로 키워드 5개 추출
feature_array = np.array(vectorizer.get_feature_names_out())
tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]

# 상위 5개 키워드 출력
top_n = 10
top_keywords = feature_array[tfidf_sorting][:top_n]
print("Top 5 Keywords:", top_keywords)



Top 5 Keywords: ['렌즈' '착용' '사용' '세균' '착용 렌즈' '동정' '미생물' '콘택트렌즈' '식염수' '일일 착용']


영어-한국어 키워드 추출

In [ ]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import PyPDF2

# KONLPY 설치 코드 (주석 처리하여 참고용으로 제공)
# !pip install konlpy

# PDF 파일에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# PDF 파일 경로 설정
pdf_path = '/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/피부암분류.pdf'

# PDF 파일에서 텍스트 추출
korean_text = extract_text_from_pdf(pdf_path)

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    return text

# 형태소 분석 및 명사 추출 함수
def extract_nouns_and_english(text):
    okt = Okt()
    nouns = okt.nouns(text)
    # 영어 단어도 포함하기 위해 정규 표현식을 사용해 영어 단어 추출
    english_words = re.findall(r'[a-zA-Z]+', text)
    return [' '.join(nouns + english_words)]

# 텍스트 전처리
preprocessed_text = preprocess_text(korean_text)

# 명사 및 영어 단어 추출
nouns_text = extract_nouns_and_english(preprocessed_text)

# TF-IDF 벡터라이저 초기화 및 키워드 추출 (ngram_range=(1, 2)로 설정하여 unigram과 bigram을 모두 고려)
# 불용어(stop words) 제거를 위해 stop_words 파라미터 설정
stop_words = ['of', 'and', 'the', 'in', 'to', 'for', 'on', 'with', 'at', 'by', 'an', 'be', 'this', 'is']
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(nouns_text)

# TF-IDF 결과를 가져와서 높은 순으로 키워드 5개 추출
feature_array = np.array(vectorizer.get_feature_names_out())
tfidf_scores = tfidf_matrix.toarray().flatten()

# TF-IDF 가중치를 기준으로 정렬
tfidf_sorting = np.argsort(tfidf_scores)[::-1]

# 중복된 키워드 제거 및 상위 5개 키워드 추출
seen_keywords = set()
top_keywords = []
for idx in tfidf_sorting:
    keyword = feature_array[idx]
    if keyword not in seen_keywords:
        top_keywords.append(keyword)
        seen_keywords.add(keyword)
    if len(top_keywords) >= 10:
        break

# 상위 5개 키워드 출력
print("Top 5 Keywords:", top_keywords)

# 결과 예시 출력
top_keywords

Top 5 Keywords: ['모델', '데이터', 'attention', '성능', '이미지', '학습', '분류', 'layer', '클래스', '기법']


['모델', '데이터', 'attention', '성능', '이미지', '학습', '분류', 'layer', '클래스', '기법']